In [1]:
using Plots
include("datasetloader.jl")
include("ngrams.jl")

load_vocab (generic function with 1 method)

In [2]:
function norm_table!(t)
    D = last(t)
    vs = sum(values(D))
    for (k, v) in D
        D[k] /= vs
        D[k] = log(D[k])
    end
end

norm_table! (generic function with 1 method)

In [142]:
GS = Dict()
for lang in LANGS
    GS[lang] = load_ngrams(joinpath("ngrams0", lang * ".txt"))
end
G_sum = sum(values(GS))
all_codes = collect(keys(G_sum));
G_avg = (1/50)*G_sum

Dict{Tuple{UInt8, Vararg{UInt8}}, Float64} with 910697 entries:
  (0xb6, 0x6c, 0x76, 0x20)       => 2.51294e-6
  (0x6f, 0x20, 0x6b, 0x65, 0x73) => 1.48226e-6
  (0x74, 0x65, 0x64, 0x20, 0x63) => 3.7827e-6
  (0x20, 0x74, 0x75, 0x6c, 0x6d) => 4.02394e-7
  (0x6e, 0x65, 0x72, 0x77, 0x6f) => 2.03344e-6
  (0x74, 0x20, 0x62, 0x6c)       => 0.00012263
  (0x6f, 0x62, 0x65, 0x79, 0x65) => 1.14394e-7
  (0x6d, 0x20, 0x61, 0xc5, 0xad) => 1.30035e-6
  (0x88, 0x89, 0xe8, 0xa1, 0x8c) => 4.0716e-6
  (0xa5, 0x75, 0x20)             => 3.519e-5
  (0x6a, 0x6e, 0x20, 0x61, 0x62) => 1.01613e-7
  (0x97, 0xb6, 0xe7)             => 6.96995e-6
  (0x6d, 0x20, 0x74, 0x69, 0x6f) => 1.64769e-6
  (0x6f, 0x6e, 0x73, 0x69, 0x6e) => 7.88614e-6
  (0x20, 0x64, 0x6f, 0x72, 0x66) => 1.00262e-5
  (0x82, 0x92, 0xe5, 0x90, 0x90) => 4.51904e-7
  (0x72, 0x74, 0x20, 0x6a, 0x65) => 1.74196e-6
  (0x20, 0x74, 0x65, 0x79)       => 2.84771e-6
  (0xa8, 0xd8, 0xa7, 0x20, 0xd8) => 0.000107501
  ⋮                              => ⋮

In [143]:
sort([l=>sum(values(D)) for (l, D) in GS], by=last, rev=true)

50-element Vector{Pair{String, Float64}}:
 "rus" => 35.58628331397364
 "eng" => 34.9222755314685
 "deu" => 34.27479354056816
 "ita" => 34.02995117687403
 "tur" => 33.9756367322808
 "fra" => 33.6822524184102
 "heb" => 33.557930400893184
 "spa" => 33.3947337918854
 "por" => 33.38888763544047
 "jpn" => 33.379976385875494
       ⋮
 "tat" => 29.50726455626446
 "nor" => 29.500759446814286
 "nds" => 29.47648987077519
 "vie" => 29.430768780601717
 "kor" => 29.204329962552144
 "cat" => 29.072319269071944
 "isl" => 28.894979639764063
 "slk" => 28.702952882802748
 "ido" => 28.034869375267558

In [144]:
norm_table!.(collect(GS));
norm_table!(1=>G_avg);
DEFAULT_LOGP = (minimum.(values.(last.(collect(GS))))|>minimum)


-17.203694184405627

In [145]:
code_score(code) = only(diff(extrema([get(D, code, DEFAULT_LOGP) for D in values(GS)])|>collect)) + G_avg[code]

code_score (generic function with 1 method)

In [146]:
code_std = [code => code_score(code) for code in all_codes]

910697-element Vector{Pair{A, Float64} where A}:
       (0xb6, 0x6c, 0x76, 0x20) => -11.498977207702632
 (0x6f, 0x20, 0x6b, 0x65, 0x73) => -12.858051933672316
 (0x74, 0x65, 0x64, 0x20, 0x63) => -10.991295972168448
 (0x20, 0x74, 0x75, 0x6c, 0x6d) => -15.248241707319703
 (0x6e, 0x65, 0x72, 0x77, 0x6f) => -11.9856153849424
       (0x74, 0x20, 0x62, 0x6c) => -4.9841811906508084
 (0x6f, 0x62, 0x65, 0x79, 0x65) => -17.84762515785424
 (0x6d, 0x20, 0x61, 0xc5, 0xad) => -12.934035318721964
 (0x88, 0x89, 0xe8, 0xa1, 0x8c) => -10.566316306224147
             (0xa5, 0x75, 0x20) => -6.450799493549852
                                ⋮
 (0x6e, 0x6f, 0x74, 0x73, 0x20) => -15.049200145457807
 (0x20, 0x6a, 0x6f, 0x74, 0x6b) => -8.075808444245268
 (0xdb, 0x8c, 0xd8, 0xb4, 0xdb) => -6.772509627947278
             (0x69, 0x70, 0x65) => -5.0409705589607405
       (0xab, 0x72, 0x6f, 0x6a) => -11.528706118771044
       (0x74, 0xc3, 0xb3, 0xc5) => -12.48744652794131
       (0x6f, 0x73, 0x20, 0x65) => -2.782447

In [147]:
sorted_codes = sort(code_std, by=last, rev=true)

910697-element Vector{Pair{A, Float64} where A}:
                        (0xd0,) => 9.789988328246363
                        (0xd7,) => 9.189355800444929
                        (0xe0,) => 8.540854352603098
                        (0xa4,) => 8.533526354050773
                        (0xd1,) => 8.398282134091286
                        (0xd8,) => 8.188728589837648
                   (0xe0, 0xa4) => 8.13202681884915
                   (0x20, 0xe0) => 7.97008073646502
                        (0xa6,) => 7.748458490711571
                        (0xce,) => 7.711652084296042
                                ⋮
 (0x6f, 0x61, 0x73, 0x69, 0x61) => -21.005582567753024
 (0x73, 0x65, 0x65, 0x64, 0x6c) => -21.005582567753024
 (0x62, 0x65, 0x67, 0x65, 0x74) => -21.005582567753024
 (0x68, 0x20, 0x6d, 0x6f, 0x6d) => -21.005582567753024
 (0x69, 0x72, 0x6c, 0x6f, 0x63) => -21.005582567753024
 (0x73, 0x61, 0x74, 0x20, 0x71) => -21.005582567753024
       (0x75, 0x74, 0x74, 0x68) => -21.005582567753024
 (0x

In [148]:
vocab = first.(sorted_codes[1:30000]);

In [149]:
dump_vocab(vocab, "vocab.txt")

In [150]:
vocab = Set(vocab);

In [151]:
GS = Dict()
for lang in LANGS
    GS[lang] = load_ngrams(joinpath("ngrams0", lang * ".txt"))
end

In [152]:
GSF = Dict()
for (lang, D) in GS
    DF = Dict(k=>v for (k,v) in D if k in vocab)
    GSF[lang] = DF
end

In [153]:
mkpath("ngrams")
for (lang, D) in GSF
    dump_ngrams( sort(D|>collect, by=last, rev=true), joinpath("ngrams", lang * ".txt"))
end